# Foundations of Data Science
## S1 Week 06: Linear Models

**Learning outcomes:** 
In this lab you will learn to apply linear regression to data sets and evaluate your model. By the end of the lab you will be able to 
* Apply linear regression using the statsmodels package
* Interpret some of the visual and numerical diagnostics from the package
* Transform variables to produce a better model of the data
* Apply multiple regression to some data

**Prerequisites:** We expect you to have understood the underlying theory of single and multiple linear regression, and we won't repeat it in this lab. If something is unclear check the course material.

Insulin is a hormone that regulates glucose levels in the body. In people with diabetes, the body does not produce enough insulin or doesn't respond to it normally, so glucose levels can become dangerously high. Some people with certain types of diabetes prevent their glucose levels becoming too high by injecting themselves with insulin. The first question we are going to try to answer is:

**Research question 1:** Can we predict the insulin level of a diabetic patient by measuring the glucose concentration in their bloodstream?

Insulin is typically injected in subcutaneous tissue.  However, if  incorrectly injected it can end up in muscle tissue instead. In this [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3897752/), researchers tried to understand the effects of age, sex, body mass index (BMI), and anatomical site on skin thickness. Being able to predict the skin thickness helps to find the optimal site for subcutaneous injections. We will try to answer this question using the diabetes data set from the S1 Week 4 Lab.

**Research question 2:** Can we give a good estimate of the skin thickness, given age and BMI?

**Data description:** See the S1 Week 4 lab.

**Remarks:**
- For some exercises you might need to use Google, StackOverflow or the official package documentation. It is important to get familiar with looking up how to solve problems online. Throughout your career you will encounter many problems that other people have encountered too; there is no need to reinvent the wheel.
- We will not provide all the code in the tutorials, but expect you to be able to draw from previous labs to fill in gaps.
- Try not to use copy+paste when coding these labs, as typing will help you memorize the code better.
- Try to understand each detail in the code we provide, and read the comments!

**Pair programming:** We encourage you to work in pairs again. Remember that the magic command you need to get the link for the session with your partner is:

In [ ]:
!echo "https://noteable.edina.ac.uk/user/$(jupyter notebook list | grep -oP '(?<=user\/).*(?=\/\?)' )/tree?token=$( jupyter notebook list | grep -oP '(?<=token=).*(?= ::)' )"

## A. Data preparation and exploration

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

We can now start by loading the data we will use in this lab.

**Exercise 01:** 

a) Load the following datasets:
- `diabetes.csv` from `datasets` and store as `diabetes_raw`

b) Plot a pairplot of the `diabetes_raw` dataset. Use two different colours to colour the patients with diabetes and without diabetes differently.

In [ ]:
# Your code
diabetes_loc = os.path.join(os.getcwd(), 'datasets', 'diabetes.csv')
diabetes_raw = pd.read_csv(diabetes_loc)

In [ ]:
sns.pairplot(diabetes_raw, hue='Outcome')

Insulin is a hormone helping in the regulation of glucose in your blood. It signals that excess glucose should be stored as glycogen. When your body needs energy, glucagon, a different hormone, transforms glycogen back into energy. Diabetes is characterized by a misfunction of insulin in your body. In type 1 diabetes the body cannot produce sufficient insulin, while in type 2 diabetes the body does not respond efficiently to insulin. Knowing the above, we would expect that diabetic people have a higher glucose level. Let's check it. 

If you look at the data carefully, you will see that there are a lot of data entries that are `0.0` that don't make sense biologically for BMI, skin thickness, glucose or insulin. The documentation about the dataset doesn't explain what the coding for these values are so we'll have to assume that they are missing values, and remove them. Let's clean the data as follows:

**Exercise 02:**
- Clean the data set by removing all observations where any of the BMI, skin thickness, Glucose, Insulin are zero. Save your dataset as `diabetes`.

In [ ]:
diabetes = diabetes_raw[(diabetes_raw['Glucose'] != 0) & (diabetes_raw['Insulin'] != 0) & (diabetes_raw['BMI'] != 0) & (diabetes_raw['SkinThickness'] != 0)]
sns.pairplot(diabetes, hue='Outcome')

**Exercise 03:** Let us zoom into the Glucose distributions. Use Seaborn to create a new figure with two overlapping distributions showing the glucose levels in diabetics and non-diabetics. Does the plot show what we expected, i.e. a higher level of Glucose for diabetics?

In [ ]:
sns.distplot(diabetes[(diabetes['Outcome']==0)]['Glucose'], label='No diabetes')
sns.distplot(diabetes[(diabetes['Outcome']==1)]['Glucose'], label='Diabetes')
plt.legend()
# Yes diabetics have a higher level of glucose on average

## B. Correlation analysis

We can get a quick impression of relationships between the variables by looking at the correlation coefficients between each pair of variables. The pandas `df.corr()` method allows us to compute correlations between all pairs of variables:

In [ ]:
diabetes.corr()

It can be helpful to visualise the correlation as a heatmap. 

**Exercise 04:**
- Plot the correlation as a heatmap, either using 2D histograms from matplotlib, or seaborn's heatmap function - an advantage of seaborn is that you can annotate the heatmap with the values of the correlation coefficient.
- Read off the correlation between Insulin and Glucose from the heatmap or the printed output above.
- Compare it to the pairplot you've just created. Is the correlation roughly what you would expect? How do we interpret this value?

In [ ]:
# Your code
# A
sns.heatmap(diabetes.corr(), annot=True)
# B
# r = 0.581
# C
# Yes, the correlation coefficient and the pairplot seem consistent and both suggest a moderate correlation.
# r is the linear correlation coefficient

## C. Linear regression

Now that we know how insulin works, we would expect the body to react with more insulin if more glucose was in the body. For this we'll use a linear regression of Insulin on Glucose.

We first need a single plot of the raw data, to compare later with our regression model.

**Exercise 05:** Create a new figure with two overlapping scatter plots, showing glucose vs insulin levels; one scatter plot for diabetic and one for non-diabetic people.

In [ ]:
# Your code
# Version 1 (Matplotlib)
fig, ax = plt.subplots()
insulin_glucose = ax.scatter(diabetes['Glucose'],
            diabetes['Insulin'],
            c=diabetes['Outcome'])
ax.set_xlabel('Glucose')
ax.set_ylabel('Insulin')
ax.legend(*insulin_glucose.legend_elements(), loc="upper left", title="Diabetes")

In [ ]:
# Version 2 (Seaborn)
sns.scatterplot(data=diabetes, hue='Outcome', y='Insulin', x='Glucose')

### C.1 Fitting the linear regression model with statmodels

We want to be able to predict the insulin level given the glucose concentration. As the output (insulin level) is continuous, we need to use a regression model. We will start by applying a linear regression model to the data.

The goal in linear regression is to compute a linear dependency between a dependent variable $y$ and a set of independent variables $\{x_1, ..., x_d\}$, such that $y = \beta_0 + \beta_1 x_1 + \beta_2 x_2+ ... + \beta_d x_d$. 

To do this we'll use the [statsmodels](https://www.statsmodels.org/l) package, "a Python module that provides classes and functions for the estimation of many different statistical models, as well as for conducting statistical tests, and statistical data exploration." There are other packages that implement linear regression - notably [scikit-learn](https://scikit-learn.org/stable/) - but the focus of sckit-learn is more on machine learning, and its linear regression routines do not perform all the statistical analysis done by statsmodels. Although we won't be using most of the information produced by statsmodels in this lab, later in the course (in Semester 2) when we come to investigate confidence intervals, it will be interesting to compare the results obtained by statsmodels, and those produced by your own coding.

Statsmodels has a special syntax for describing linear regression, which it shares with [R](https://www.r-project.org/), an important language for statistical analysis. For linear regression on one independent variable, the syntax `y ~ x` means "regress y on x". The function `statsmodels.formula.api.ols()` initialises an ordinary least squares (OLS) model, taking a regression formula, and a pandas dataframe in which the variables in the regression formula are columns. We can `fit()` this model, i.e. compute the regression coefficients, returning a `RegressionResults` object that we'll call `results`. 

In [ ]:
model = smf.ols('Insulin ~ Glucose', data=diabetes)
results = model.fit()

**Exercise 06:**

- Use the `predict()` method of `results` to generate the fitted values $\hat y$,  and store the result in `y_hat`.
- Create a plot containing the scatter plot above, and the regression line.

In [ ]:
# Your answer
sns.scatterplot(data=diabetes, hue='Outcome', y='Insulin', x='Glucose')
## x = diabetes['Glucose'] and yhat = results.predict(diabetes['Glucose'])
y_hat = results.predict(diabetes['Glucose'])
sns.lineplot(diabetes['Glucose'], y_hat)

**Discussion:** What can you observe about the fit to the data?

Your answer:
- Fit seems reasonable
- More spread of Insulin around the regression line for high values of Glucose (heteroscedasticity)

### C.2 Numerical diagnostics

We can generate a lot of information about the regression analysis by using the `summary()` method:

In [ ]:
results.summary()

Here we should recognise some of the things we saw in the video lectures: the coefficient of determination (`R-squared`) and adjusted coefficient of determination (`Adj. R-squared`) and also the `Intercept` and `Glucose` `coef`s (in the second part of the table) - these correspond to $\hat\beta_0$ and $\hat\beta_1$ in the linear regression model. There are lots of other measures in here; we won't worry about these for the moment.

**Discussion:** What are the values of the adjusted and unadjusted coefficients of determination and the coefficients? Are the coefficients consistent with the hypothesis that higher glucose predicts higher insulin?

Your answer:
- R2 = 0.338
- R2a = 0.336
- beta0 = -118.41
- beta1 = 2.2382

- Yes the coefficient $\beta_1$ indicates that for every 1 unit of glucose, insulin is 2.24 units higher.

One thing that is missing from the summary to evaluate our model is the mean squared error (MSE) and the root mean squared error (RMSE), but we can find the MSE from the `mse_resid` property of the results:

In [ ]:
MSE = results.mse_resid
RMSE = np.sqrt(MSE)
print('Mean Squared Error:', MSE)  
print('Root Mean Squared Error:', RMSE)

**Discussion:** Does the RMSE look to match the scale of the deviations of data points from the regression line in the figure you've just plotted?

Your answer:
- Looking at the plot, the spread around the regression line looks to be on a scale of around 100.

### C.3 Visual diagnostics

We'll now plot a residual plot as shown in the lecture.

In [ ]:
fig, ax = plt.subplots()
ax.set_ylabel('Residual')
ax.set_title('Residual versus independent variable (Glucose)')
scatter = sns.scatterplot(diabetes['Glucose'], results.resid)

In [ ]:
fig, ax = plt.subplots()
ax.set_ylabel('Residual')
ax.set_xlabel('Fitted value of Insulin')
ax.set_title('Residual versus fitted (predicted) variable (Insulin)')
scatter = sns.scatterplot(y_hat, results.resid)

**Discussion:** Which one of the two properties of residual plots shown in the lecture notes are these residuals showing?

Your answer:

Heteroscedasticity

### C.4 Linear regression with transformed variables

Let's see if we can make the plot less heteroscedastic. The problem with the second residual plot is that for smaller values of the dependent variable $y$ (`Insulin`), the residuals are smaller than for larger values of the dependent variable $y$. Taking the log of `Insulin` will tend to amplify the differences for small values of $y$ and reduce the differences for large values of $y$. We'll therefore try a log transform of `Insulin`, and fit a new model, which we'll call the `logmodel`. The first step is to create a column containing the log transform `Insulin`:

In [ ]:
diabetes.insert(loc = len(diabetes.columns), column = 'LogInsulin', value = diabetes['Insulin'].apply(np.log10))
diabetes.head()

Then we fit and plot the log model as above:

In [ ]:
logmodel = smf.ols('LogInsulin ~ Glucose', data=diabetes)
logresults = logmodel.fit()

In [ ]:
sns.scatterplot(data=diabetes, hue='Outcome', y='LogInsulin', x='Glucose')
sns.lineplot(diabetes['Glucose'], logresults.predict(diabetes['Glucose'])) # x = diabetes['Glucose'] and yhat = results.predict(diabetes['Glucose'])

That looks like a nicer fit, with a more even spread of the data points around the regression line. Do the residuals confirm this impression?

In [ ]:
sns.scatterplot(diabetes['Glucose'], logresults.resid)

Yes! The variance of the residuals doesn't seem to vary much with the level of Glucose - definitely less heteroscedastic. Fantastic! The residuals do perhaps appear to have a bit of a longer negative tail - the minimum is around -1.25 and the maximum is 0.75.

Now let's look at the numerical diagnostics.

In [ ]:
logresults.summary()

**Discussion:**
1. How does the coefficient of determination and adjusted coefficient of determination compare to the original (untransformed) fit?
2. What are the coefficients here?

Your answer:

- R2 = 0.38 rather than 0.338
- R2a = 0.379  rather than 0.336
- Beta0 = 1.3500
- Beta1 = 0.0060

The regression equation of $\log_{10} y$ on $x$ is:

$\log_{10} y = \beta_0 + \beta_1 x$

**Exercise 07:**

1. Rearrange this equation to give an equation for $y$ in terms of $x$
2. Now substitute in the coefficients found by the linear regression analysis above to give an equation for Insulin in terms of Glucose.

Your answer:

`LogInsulin = 1.3502 + 0.0060*Glucose`

`Insulin = 10^(1.3502)*10^(0.0060*Glucose)`

**Exercise 08:** Plot this function against the original linear fit:

In [ ]:
sns.scatterplot(data=diabetes, hue='Outcome', y='Insulin', x='Glucose')
sns.lineplot(diabetes['Glucose'], results.predict(diabetes['Glucose'])) # x = diabetes['Glucose'] and yhat = results.predict(diabetes['Glucose'])
sns.lineplot(diabetes['Glucose'], np.power(10, 1.3502 + 0.0060*diabetes['Glucose']))

We can see the fit is similar.

There are other metrics in the summary information that indicate that this is a better fit - but we will return to this issue later in the course.

The rest of the lab is optional, however, we encourage you to at least try the first part on multiple regression, which is very short, as it gives you a deeper understanding of linear regression.

## D. Optional exercises

### D.1 Multiple regression

In this [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3897752/) researchers tried to figure out the factors that influenced skin thickness, in order to figure out the optimal depth to inject insulin to diabetes patients. They considered age, BMI, sex and anatomical site (i.e. where in the body). If we look at the plots from Exercise 01, we can already see that there seems to be a linear dependency between skin thickness and the BMI. However, let us inspect the data more closely.

**Exercise 09:**

Create a new figure with two scatter plots (seaborn or matplotlib, your choice). One should depict BMI vs skin thickness and the other age vs skin thickness.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.scatter(diabetes['BMI'], diabetes['SkinThickness'])
ax2.scatter(diabetes['Age'], diabetes['SkinThickness'])
ax1.set_ylabel('Skin Thickness')
ax1.set_xlabel('BMI')
ax2.set_xlabel('Age')

**Discussion:** Do you expect that linear regression will be a good model?

Your answer:
- It looks like there is a clear linear dependency between BMI and skin thickness. However, age does not seem to have a clear linear dependency with skin thickness. Thus, we can expect that the BMI will give a meaningful prediction but that the age won't add much information to the model.

**Exercise 10:** Create a new instance of a linear regression model of SkinThickness on BMI, and compute the $R^2$ score between the predicted values and the actual values.

In [ ]:
# Your code
model1 = smf.ols('SkinThickness ~ BMI', data=diabetes)
results1 = model1.fit()
results1.summary()

Above, we have said that in the paper, the researchers tried to compute the effect of age, BMI, sex and anatomical site on the skin thickness. We only have the age and BMI. However, we can try to compute a 2-D linear regression.

- The syntax for regressing $y$ on $x_1$ and $x_2$ in statsmodels is:

`y ~ x1 + x2`

**Exercise 11:**
- Fit the multiple regression model (SkinThickness on Age and BMI)
- Compute the metrics for the new model.
- Did our model improve?

In [ ]:
model2 = smf.ols('SkinThickness ~ BMI + Age', data=diabetes)
results2 = model2.fit()
results2.summary()

**Discussion:** 

- Is our conjecture that Age doesn't have a strong influence on the skinthickness correct?

Your answer:

The R^2 has increased slightly, but not by a great deal, suggesting that Age as an untransformed variable does not have a great influence on the skin thickness.

### D.2 A log-log model

**Exercise 12:** In the regression of log Insulin on Glucose, you can try log-transforming the independent variable (Glucose) too.

In [ ]:
diabetes2 = diabetes.copy()
diabetes2['LogGlucose'] =  diabetes['Glucose'].apply(np.log10)

diabetes = diabetes2

loglogmodel = smf.ols('LogInsulin ~ LogGlucose', data=diabetes)
loglogresults = loglogmodel.fit()

In [ ]:
sns.scatterplot(data=diabetes, hue='Outcome', y='LogInsulin', x='LogGlucose')
sns.lineplot(diabetes['LogGlucose'], loglogresults.predict(diabetes['LogGlucose'])) # x = diabetes['Glucose'] and yhat = results.predict(diabetes['Glucose'])

In [ ]:
sns.scatterplot(diabetes['LogGlucose'], loglogresults.resid)

In [ ]:
loglogresults.summary()

### D.3 Coding up regression

**Exercise 13:**. If you're curious to understand the implementation of regression, and have the time, try to implement functions `beta0, beta1 = regression(x, y)`, which returns the coefficients $\hat\beta_0$ and $\hat\beta_1$ from the data, and `yhat = predict(x, beta0, beta1)` which returns the fitted value $\hat y$. You can check it against the results from `statsmodels`.

In [ ]:
## Your answer: